In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-987dzqin
  Running command git clone --filter=blob:none --quiet https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-987dzqin
  Resolved https://github.com/ssut/py-hanspell.git to commit fdc6ca50c19f1c85971437a072d89d4e5ce024b8
  Preparing metadata (setup.py) ... done
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4812 sha256=f89052adecc1995758cf43526a35c9175b190dd862fd700f47d3c9835f365768
  Stored in directory: /tmp/pip-ephem-wheel-cache-sr75kk4y/wheels/18/0f/39/d0e3972de3368ba3ab62817cd7b17a74123414afd774ce604b
Successfully built py-hanspell


In [ ]:
!pip install emoji

In [ ]:
import pandas as pd

# 전처리 예시

### 영양정보 추출

In [ ]:
lactor_df = pd.read_csv("/content/drive/MyDrive/데이터 수집/1. 데이터 통합/네이버/네이버_유산균_리뷰_합본.csv")
lactor_df.head()

,NO,product_name,Content,Info,category
0,NaN,Meditamin,뚜껑이 살짝 불편한 거 빼고는 조아요 ! 효과있어요 먹고 화장실 잘갑니다!!,상품정보신고하기\n상품번호 5233716298 상품상태 신상품\n제조사 알피바이오 ...,유산균
1,NaN,Meditamin,가성비 좋고 먹기 편해요 이제 여기꺼로 정착할듯 합니다,상품정보신고하기\n상품번호 5233716298 상품상태 신상품\n제조사 알피바이오 ...,유산균
2,NaN,Meditamin,포장 꼼꼼하게 빨리 배송되었어요... 효과는 좋은지 꾸준히 먹어봐야 알것 같아요...,상품정보신고하기\n상품번호 5233716298 상품상태 신상품\n제조사 알피바이오 ...,유산균
3,NaN,Meditamin,먹기도 간편하고 약통이 작아서 자리도 많이 안차지해서 좋고 무엇보다 만성변비인데 서...,상품정보신고하기\n상품번호 5233716298 상품상태 신상품\n제조사 알피바이오 ...,유산균
4,NaN,Meditamin,항상 잘 먹고잇네요,상품정보신고하기\n상품번호 5233716298 상품상태 신상품\n제조사 알피바이오 ...,유산균


In [ ]:
lactor_df["Info"][0]

'상품정보신고하기\n상품번호 5233716298 상품상태 신상품\n제조사 알피바이오 브랜드 메디타민\n모델명 대장건강 비피더스 프로바이오틱스 100억 500mg x 30캡슐 이벤트 정기구독이벤트 오후3시까지 당일 발송 및 무료배송\n원산지 상세설명에 표시\n상품정보 관련 문의사항은 Q&A에 남겨주세요.\n제품타입 캡슐 섭취방법 물과 함께\n섭취대상 성인남녀 섭취횟수 하루 한 번\n1일 총 섭취량 1캡슐 제품용량 3개월분\n주요 기능성(식약처인증) 장건강 영양소 원료명(식약처고시) 해당 없음\n생균 100억 CFU\n상품정보\n영수증발급\n신용카드전표, 현금영수증발급\nA/S 안내\n상세정보 확인\n인증정보\n건강기능식품 광고사전심의_국가인증 - (사)한국건강기능식품협회 표시·광고심의위원회\n230411323\n구매전 인증번호를 꼭 확인하세요.\n해당 인증정보는 판매자가 등록한 것으로 등록 정보에 대한 일체의 책임은 판매자에게 있습니다.'

In [ ]:
# 기능성 블록 추출 함수
def extract_function_block(text):
    if pd.isna(text):
        return None
    # 주요 기능성부터 상품정보 전까지
    match = re.search(r"(주요\s*기능성.*?)(?=상품정보)", text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return None

# apply로 전체 행에 적용 (벡터화처럼 작동)
lactor_df["Nutri"] = lactor_df["Info"].apply(extract_function_block)

In [ ]:
lactor_df["Nutri"][4356]

'주요 기능성(식약처인증) 장건강 영양소 원료명(식약처고시) 해당 없음\n생균 100억 CFU'

### 리뷰 데이터 전처리(특수문자, 이모지 등 제거)

In [ ]:
tqdm.pandas()

# 이모지 제거
def remove_emoji(text):
  return emoji.replace_emoji(text, replace = "")

# 특수문자 제거(한글, 영어, 숫자, 공백만 유지)
def remove_special(text):
  return re.sub(r"[^가-힣a-zA-Z0-9\s]", "", text)

# 반복 문자 정규화
def normalize_repetition(text):
  return re.sub(r"(.)\1{2,}", r"\1\1", text)

# 오타 교정
def correct_korean(text):
  try:
    spelled = spell_checker.check(text)
    return spelled.checked

  except:
    return text

# 전체 전처리 함수
def preprocessing_text(text):
  if pd.isna(text):
    return ""

  text = remove_emoji(text)
  text = remove_special(text)
  text = normalize_repetition(text)
  text = correct_korean(text)
  return text.strip()

In [ ]:
lactor_df["preprocessing_cnotent"] = lactor_df["Content"].progress_apply(preprocessing_text)

  3%|▎         | 380/15000 [01:21<57:26,  4.24it/s]

In [ ]:
lactor_df.head()

In [ ]:
lactor_df[["preprocessing_cnotent", "Content"]]

# 네이버, 아마존 데이터 통합
### 1. 아마존 데이터 통합
### 2. 네이버 데이터 통합
### 3. 아마존 데이터(영어) 전처리
### 4. 네이버 데이터 전처리
### 5. 네이버 + 아마존 데이터 통합

### 아마존 데이터 통합

In [ ]:
protein_df = pd.read_csv("/content/drive/MyDrive/데이터 수집/1. 데이터 통합/아마존/아마존_단백질_리뷰_합본.csv")
protein_df.head()

,ASIN,Product_title,Body,Info,category
0,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...","Okay, so I've been incorporating these Premier...","Packaged in a 11.5oz bottle, each Premier Prot...",단백질
1,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",Getting a little older and getting that protei...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
2,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...","I have to say, this is one of the best tasting...","Packaged in a 11.5oz bottle, each Premier Prot...",단백질
3,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",Product was delivered quickly but main box and...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
4,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",I really like the taste of Premier protein ove...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질


In [ ]:
protein_df.columns = ["asin", "title", "Body", "Info", "category"]
protein_df.head()

,asin,title,Body,Info,category
0,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...","Okay, so I've been incorporating these Premier...","Packaged in a 11.5oz bottle, each Premier Prot...",단백질
1,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",Getting a little older and getting that protei...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
2,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...","I have to say, this is one of the best tasting...","Packaged in a 11.5oz bottle, each Premier Prot...",단백질
3,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",Product was delivered quickly but main box and...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
4,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",I really like the taste of Premier protein ove...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질


In [ ]:
vitamin_df = pd.read_csv("/content/drive/MyDrive/데이터 수집/1. 데이터 통합/아마존/아마존_비타민_리뷰_합본.csv")
vitamin_df.head()

,asin,title,Body,Info,category
0,B00BKNSV2A,Metagenics PhytoMulti Multivitamin Without Iro...,I’ve been taking these vitamins for over 5 yea...,"[""As the #1 Doctor Recommended Professional Su...",비타민
1,B00BKNSV2A,Metagenics PhytoMulti Multivitamin Without Iro...,Multivitamin was as advertised and a good qual...,"[""As the #1 Doctor Recommended Professional Su...",비타민
2,B00BKNSV2A,Metagenics PhytoMulti Multivitamin Without Iro...,Quality vitamins. Price is comparable to other...,"[""As the #1 Doctor Recommended Professional Su...",비타민
3,B00BKNSV2A,Metagenics PhytoMulti Multivitamin Without Iro...,"These are great , easy to swallow , give you e...","[""As the #1 Doctor Recommended Professional Su...",비타민
4,B00BKNSV2A,Metagenics PhytoMulti Multivitamin Without Iro...,"good products, come with many what we need","[""As the #1 Doctor Recommended Professional Su...",비타민


In [ ]:
zinc_df = pd.read_csv("/content/drive/MyDrive/데이터 수집/1. 데이터 통합/아마존/아마존_아연_리뷰_합본.csv")
zinc_df.head()

,asin,title,Body,Info,category
0,B0000DJAPU,"Nature Made Zinc 30 mg, Dietary Supplement for...",Nature Made Zinc Tablets are a reliable way to...,Product Description\nNature Made Zinc 30 mg Ta...,아연
1,B0000DJAPU,"Nature Made Zinc 30 mg, Dietary Supplement for...","I have been using this ""Nature Made Zinc 30 mg...",Product Description\nNature Made Zinc 30 mg Ta...,아연
2,B0000DJAPU,"Nature Made Zinc 30 mg, Dietary Supplement for...","""Nature Made Zinc 30 mg is an exceptional supp...",Product Description\nNature Made Zinc 30 mg Ta...,아연
3,B0000DJAPU,"Nature Made Zinc 30 mg, Dietary Supplement for...","Simple, effective, and affordable. I've been t...",Product Description\nNature Made Zinc 30 mg Ta...,아연
4,B0000DJAPU,"Nature Made Zinc 30 mg, Dietary Supplement for...",A quality product that I utilize on a daily ba...,Product Description\nNature Made Zinc 30 mg Ta...,아연


In [ ]:
omega_df = pd.read_csv("/content/drive/MyDrive/데이터 수집/1. 데이터 통합/아마존/아마존_오메가3_리뷰_합본.csv")
omega_df.head()

,asin,title,Body,Info,category
0,B000FL21W4,"Solgar 1300 mg Omega 3-6-9, 120 Softgels - Fis...","good product, comes in a glass bottle which is...","[""Blend Of Three Premium Oils; Solgar EFA 1300...",오메가3
1,B000FL21W4,"Solgar 1300 mg Omega 3-6-9, 120 Softgels - Fis...",Remained fresh for months. Didn’t have after t...,"[""Blend Of Three Premium Oils; Solgar EFA 1300...",오메가3
2,B000FL21W4,"Solgar 1300 mg Omega 3-6-9, 120 Softgels - Fis...",Good so far,"[""Blend Of Three Premium Oils; Solgar EFA 1300...",오메가3
3,B000FL21W4,"Solgar 1300 mg Omega 3-6-9, 120 Softgels - Fis...",Muy util,"[""Blend Of Three Premium Oils; Solgar EFA 1300...",오메가3
4,B000FL21W4,"Solgar 1300 mg Omega 3-6-9, 120 Softgels - Fis...",Supplements are tough to rate but I hear this ...,"[""Blend Of Three Premium Oils; Solgar EFA 1300...",오메가3


In [ ]:
lactor_df = pd.read_csv("/content/drive/MyDrive/데이터 수집/1. 데이터 통합/아마존/아마존_유산균_리뷰_합본.csv")
lactor_df.head()

,asin,title,Body,Info,category
0,B00L1GHMN0,"NOW Foods Supplements, Probiotic-10™, 25 Billi...",After years of being a loyal Bio-Kult customer...,10 Probiotic Strains Clinically Validated Stra...,유산균
1,B00L1GHMN0,"NOW Foods Supplements, Probiotic-10™, 25 Billi...",Love NOW products! I have to take probiotics f...,10 Probiotic Strains Clinically Validated Stra...,유산균
2,B00L1GHMN0,"NOW Foods Supplements, Probiotic-10™, 25 Billi...",Love these probiotic supplements. Can’t speak ...,10 Probiotic Strains Clinically Validated Stra...,유산균
3,B00L1GHMN0,"NOW Foods Supplements, Probiotic-10™, 25 Billi...",I liked it. Lasted me awhile.,10 Probiotic Strains Clinically Validated Stra...,유산균
4,B00L1GHMN0,"NOW Foods Supplements, Probiotic-10™, 25 Billi...",Great probiotic that I take every morning.,10 Probiotic Strains Clinically Validated Stra...,유산균


In [ ]:
amazon_df = pd.concat([protein_df, vitamin_df, zinc_df, omega_df, lactor_df])
print(len(amazon_df))

62860


In [ ]:
amazon_df = amazon_df.reset_index()

In [ ]:
amazon_df

,index,asin,title,Body,Info,category
0,0,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...","Okay, so I've been incorporating these Premier...","Packaged in a 11.5oz bottle, each Premier Prot...",단백질
1,1,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",Getting a little older and getting that protei...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
2,2,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...","I have to say, this is one of the best tasting...","Packaged in a 11.5oz bottle, each Premier Prot...",단백질
3,3,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",Product was delivered quickly but main box and...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
4,4,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",I really like the taste of Premier protein ove...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
...,...,...,...,...,...,...
62855,11867,B0811P73BT,Wholesome Wellness Organic Raw Probiotics 100 ...,I’m not able to take this particular probiotic...,"[""100 Billion CFUs, Shelf Stable, 34 Strains: ...",유산균
62856,11868,B0811P73BT,Wholesome Wellness Organic Raw Probiotics 100 ...,I had taken many different brands of probiotic...,"[""100 Billion CFUs, Shelf Stable, 34 Strains: ...",유산균
62857,11869,B0811P73BT,Wholesome Wellness Organic Raw Probiotics 100 ...,I received my bottle today and discovered I wa...,"[""100 Billion CFUs, Shelf Stable, 34 Strains: ...",유산균
62858,11870,B0811P73BT,Wholesome Wellness Organic Raw Probiotics 100 ...,Don't waste your money! I was using a NatureWi...,"[""100 Billion CFUs, Shelf Stable, 34 Strains: ...",유산균


In [ ]:
amazon_df.to_csv("/content/drive/MyDrive/데이터 수집/1. 데이터 통합/아마존/amazon_df.csv", index = False)

### 네이버 데이터 통합

In [ ]:
protein_df = pd.read_csv("/content/drive/MyDrive/데이터 수집/1. 데이터 통합/네이버/네이버_단백질_리뷰_합본.csv")
protein_df.head()

,NO,product_name,Content,Info,category
0,1.158627e+10,Bene,초코맛 사려다가 조금 더 저렴해서 샀는데 생각보다 맛있습니다. 물에 타먹었는데도 맛...,상품정보신고하기\n상품번호 11586271837 상품상태 신상품\n제조사 이선 브랜...,단백질 쉐이크
1,1.158627e+10,Bene,맛있고 먹기 좋습니다,상품정보신고하기\n상품번호 11586271837 상품상태 신상품\n제조사 이선 브랜...,단백질 쉐이크
2,1.158627e+10,Bene,운동 전후 먹으면 든든하고 좋습니다.,상품정보신고하기\n상품번호 11586271837 상품상태 신상품\n제조사 이선 브랜...,단백질 쉐이크
3,1.158627e+10,Bene,양이 많아서 좋아요 맛도 있어요.,상품정보신고하기\n상품번호 11586271837 상품상태 신상품\n제조사 이선 브랜...,단백질 쉐이크
4,1.158627e+10,Bene,만족합니다 감사합니다,상품정보신고하기\n상품번호 11586271837 상품상태 신상품\n제조사 이선 브랜...,단백질 쉐이크


In [ ]:
vitamin_df = pd.read_csv("/content/drive/MyDrive/데이터 수집/1. 데이터 통합/네이버/네이버_비타민_리뷰_합본.csv")
vitamin_df.head()

,NO,product_name,Content,Info,category
0,1.010205e+10,Sentroom_woman,날짜가 26년까지네요 실망입니다,상품정보신고하기\n상품번호 10102054620 상품상태 신상품\n제조사 콜마비앤에...,비타민
1,1.010205e+10,Sentroom_woman,50에 딱 맞기를 바라며 구매해봅니다,상품정보신고하기\n상품번호 10102054620 상품상태 신상품\n제조사 콜마비앤에...,비타민
2,1.010205e+10,Sentroom_woman,처음구입합니다 잘챙겨먹구 건강관리 잘할께요,상품정보신고하기\n상품번호 10102054620 상품상태 신상품\n제조사 콜마비앤에...,비타민
3,1.010205e+10,Sentroom_woman,먹기도 좋고 특별한 맛이 없어서 좋아요,상품정보신고하기\n상품번호 10102054620 상품상태 신상품\n제조사 콜마비앤에...,비타민
4,1.010205e+10,Sentroom_woman,항상 주문하고 있어요 잘 받았습니다 감사합니다,상품정보신고하기\n상품번호 10102054620 상품상태 신상품\n제조사 콜마비앤에...,비타민


In [ ]:
zinc_df = pd.read_csv("/content/drive/MyDrive/데이터 수집/1. 데이터 통합/네이버/네이버_아연_리뷰_합본.csv")
zinc_df.head()

,NO,product_name,Content,Info,category
0,4.652122e+09,Aanapharm,아이가 잘 챙겨먹고있어요,상품정보신고하기\n상품번호 4652121574 상품상태 신상품\n제조사 바이오로제트...,아연
1,4.652122e+09,Aanapharm,꾸준히 먹이고 있는데 애가 잘 먹어요~,상품정보신고하기\n상품번호 4652121574 상품상태 신상품\n제조사 바이오로제트...,아연
2,4.652122e+09,Aanapharm,역시 먹어줘야해요 ..ㅎㅎ 아이들이 먹기도 편하고 좋아요 ㅎ,상품정보신고하기\n상품번호 4652121574 상품상태 신상품\n제조사 바이오로제트...,아연
3,4.652122e+09,Aanapharm,꾸준히 먹이고 있어요,상품정보신고하기\n상품번호 4652121574 상품상태 신상품\n제조사 바이오로제트...,아연
4,4.652122e+09,Aanapharm,애들이 잘 먹어서 좋습니다.,상품정보신고하기\n상품번호 4652121574 상품상태 신상품\n제조사 바이오로제트...,아연


In [ ]:
omega_df = pd.read_csv("/content/drive/MyDrive/데이터 수집/1. 데이터 통합/네이버/네이버_오메가3_리뷰_합본.csv")
omega_df.head()

,NO,product_name,Content,Info,category
0,8.671227e+09,Drlean,일찍도착했고 모두 적합합니다~ 잘먹겠습니다~,상품정보신고하기\n상품번호 8671226877 상품상태 신상품\n제조사 노바렉스 브...,오메가3
1,8.671227e+09,Drlean,잘 받았어요 감사합니당,상품정보신고하기\n상품번호 8671226877 상품상태 신상품\n제조사 노바렉스 브...,오메가3
2,8.671227e+09,Drlean,좋습니다 잘먹고 있어요~,상품정보신고하기\n상품번호 8671226877 상품상태 신상품\n제조사 노바렉스 브...,오메가3
3,8.671227e+09,Drlean,운동하는 조카 선물로 구매 했어요,상품정보신고하기\n상품번호 8671226877 상품상태 신상품\n제조사 노바렉스 브...,오메가3
4,8.671227e+09,Drlean,건강보조제품 꾸준히 먹고 있네요,상품정보신고하기\n상품번호 8671226877 상품상태 신상품\n제조사 노바렉스 브...,오메가3


In [ ]:
lactor_df = pd.read_csv("/content/drive/MyDrive/데이터 수집/1. 데이터 통합/네이버/네이버_유산균_리뷰_합본.csv")
lactor_df.head()

,NO,product_name,Content,Info,category
0,NaN,Meditamin,뚜껑이 살짝 불편한 거 빼고는 조아요 ! 효과있어요 먹고 화장실 잘갑니다!!,상품정보신고하기\n상품번호 5233716298 상품상태 신상품\n제조사 알피바이오 ...,유산균
1,NaN,Meditamin,가성비 좋고 먹기 편해요 이제 여기꺼로 정착할듯 합니다,상품정보신고하기\n상품번호 5233716298 상품상태 신상품\n제조사 알피바이오 ...,유산균
2,NaN,Meditamin,포장 꼼꼼하게 빨리 배송되었어요... 효과는 좋은지 꾸준히 먹어봐야 알것 같아요...,상품정보신고하기\n상품번호 5233716298 상품상태 신상품\n제조사 알피바이오 ...,유산균
3,NaN,Meditamin,먹기도 간편하고 약통이 작아서 자리도 많이 안차지해서 좋고 무엇보다 만성변비인데 서...,상품정보신고하기\n상품번호 5233716298 상품상태 신상품\n제조사 알피바이오 ...,유산균
4,NaN,Meditamin,항상 잘 먹고잇네요,상품정보신고하기\n상품번호 5233716298 상품상태 신상품\n제조사 알피바이오 ...,유산균


In [ ]:
naver_df = pd.concat([protein_df, vitamin_df, zinc_df, omega_df, lactor_df])
print(len(naver_df))

67849


In [ ]:
naver_df

In [ ]:
naver_df.to_csv("/content/drive/MyDrive/데이터 수집/1. 데이터 통합/네이버/naver_df.csv", index = False)

### 아마존 데이터 전처리

In [ ]:
amazon_df = pd.read_csv("/content/drive/MyDrive/데이터 수집/1. 데이터 통합/아마존/amazon_df.csv")
amazon_df.head()

,index,asin,title,Body,Info,category
0,0,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...","Okay, so I've been incorporating these Premier...","Packaged in a 11.5oz bottle, each Premier Prot...",단백질
1,1,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",Getting a little older and getting that protei...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
2,2,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...","I have to say, this is one of the best tasting...","Packaged in a 11.5oz bottle, each Premier Prot...",단백질
3,3,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",Product was delivered quickly but main box and...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
4,4,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",I really like the taste of Premier protein ove...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질


In [ ]:
!pip install emoji autocorrect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for autocorrect: filename=autocorrect-2.6.1-py3-none-any.whl size=622364 sha256=b3d8b7c93ac5477a6f6014ebac66de6e988d19c66b0c3ba3c775a11b8ece3d5a
  Stored in directory: /root/.cache/pip/wheels/5e/90/99/807a5ad861ce5d22c3c299a11df8cba9f31524f23ae6e645cb
Successfully built autocorrect


In [ ]:
import re
import emoji
import nltk

from autocorrect import Speller
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize
from spellchecker import SpellChecker

In [ ]:
amazon_df = amazon_df.dropna()

In [ ]:
amazon_df.isnull().sum()

,0
index,0
asin,0
title,0
Body,0
Info,0
category,0


In [ ]:
nltk.download("punkt_tab")
nltk.download("wordnet")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
spell = Speller(lang='en')

In [ ]:
def normalize_repeating_characters(text):
    return re.sub(r'(.)\1{2,}', r'\1\1', text)

In [ ]:
# 전처리 함수
def preprocess_text(text):
    if not isinstance(text, str):
        return ""

    # 1. 이모지 제거
    text = emoji.replace_emoji(text, replace='')

    # 2. 소문자 변환
    text = text.lower()

    # 3. 특수문자 제거 (영어, 숫자, 공백만 유지)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # 4. 반복 문자 정규화
    text = normalize_repeating_characters(text)

    # 5. 오타 교정
    text = spell(text)

    # 6. 여백 정리
    text = re.sub(r'\s+', ' ', text).strip()

    return text


In [ ]:
from tqdm import tqdm

tqdm.pandas()
amazon_df["preprocessing"] = amazon_df["Body"].progress_apply(preprocess_text)

100%|██████████| 62748/62748 [2:13:24<00:00,  7.84it/s]


In [ ]:
amazon_df = amazon_df[["asin", "title", "preprocessing", "Info", "category"]]

In [ ]:
amazon_df.columns = [["Product_no", "Product_name", "Content", "Product_info", "category"]]

In [ ]:
amazon_df.head()

,Product_no,Product_name,Content,Product_info,category
0,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",okay so ive been incorporating these premier p...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
1,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",getting a little older and getting that protei...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
2,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",i have to say this is one of the best tasting ...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
3,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",product was delivered quickly but main box and...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
4,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",i really like the taste of premier protein ove...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질


In [ ]:
amazon_df.to_csv("/content/drive/MyDrive/데이터 수집/2.데이터 전처리/Preprocessing_amazon.csv", index = False)

### 네이버 데이터 전처리

In [ ]:
naver_df = pd.read_csv("/content/drive/MyDrive/데이터 수집/1. 데이터 통합/네이버/naver_df.csv")
naver_df.head()

,NO,product_name,Content,Info,category
0,1.158627e+10,Bene,초코맛 사려다가 조금 더 저렴해서 샀는데 생각보다 맛있습니다. 물에 타먹었는데도 맛...,상품정보신고하기\n상품번호 11586271837 상품상태 신상품\n제조사 이선 브랜...,단백질 쉐이크
1,1.158627e+10,Bene,맛있고 먹기 좋습니다,상품정보신고하기\n상품번호 11586271837 상품상태 신상품\n제조사 이선 브랜...,단백질 쉐이크
2,1.158627e+10,Bene,운동 전후 먹으면 든든하고 좋습니다.,상품정보신고하기\n상품번호 11586271837 상품상태 신상품\n제조사 이선 브랜...,단백질 쉐이크
3,1.158627e+10,Bene,양이 많아서 좋아요 맛도 있어요.,상품정보신고하기\n상품번호 11586271837 상품상태 신상품\n제조사 이선 브랜...,단백질 쉐이크
4,1.158627e+10,Bene,만족합니다 감사합니다,상품정보신고하기\n상품번호 11586271837 상품상태 신상품\n제조사 이선 브랜...,단백질 쉐이크


In [ ]:
# 기능성 블록 추출 함수
def extract_function_block(text):
    if pd.isna(text):
        return None
    # 주요 기능성부터 상품정보 전까지
    match = re.search(r"(주요\s*기능성.*?)(?=상품정보)", text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return None

# apply로 전체 행에 적용 (벡터화처럼 작동)
naver_df["Nutri"] = naver_df["Info"].apply(extract_function_block)

In [ ]:
naver_df["Nutri"]

,Nutri
0,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g
1,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g
2,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g
3,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g
4,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g
...,...
67844,주요 기능성(식약처인증) 체지방 감소 영양소 원료명(식약처고시) 크롬\n생균 100...
67845,주요 기능성(식약처인증) 체지방 감소 영양소 원료명(식약처고시) 크롬\n생균 100...
67846,주요 기능성(식약처인증) 체지방 감소 영양소 원료명(식약처고시) 크롬\n생균 100...
67847,주요 기능성(식약처인증) 체지방 감소 영양소 원료명(식약처고시) 크롬\n생균 100...


In [ ]:
naver_df.head()

,NO,product_name,Content,Info,category,Nutri
0,1.158627e+10,Bene,초코맛 사려다가 조금 더 저렴해서 샀는데 생각보다 맛있습니다. 물에 타먹었는데도 맛...,상품정보신고하기\n상품번호 11586271837 상품상태 신상품\n제조사 이선 브랜...,단백질 쉐이크,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g
1,1.158627e+10,Bene,맛있고 먹기 좋습니다,상품정보신고하기\n상품번호 11586271837 상품상태 신상품\n제조사 이선 브랜...,단백질 쉐이크,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g
2,1.158627e+10,Bene,운동 전후 먹으면 든든하고 좋습니다.,상품정보신고하기\n상품번호 11586271837 상품상태 신상품\n제조사 이선 브랜...,단백질 쉐이크,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g
3,1.158627e+10,Bene,양이 많아서 좋아요 맛도 있어요.,상품정보신고하기\n상품번호 11586271837 상품상태 신상품\n제조사 이선 브랜...,단백질 쉐이크,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g
4,1.158627e+10,Bene,만족합니다 감사합니다,상품정보신고하기\n상품번호 11586271837 상품상태 신상품\n제조사 이선 브랜...,단백질 쉐이크,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g


In [ ]:
from spellchecker import SpellChecker
spell = SpellChecker()

In [ ]:
tqdm.pandas()

# 이모지 제거
def remove_emoji(text):
  return emoji.replace_emoji(text, replace = "")

# 특수문자 제거(한글, 영어, 숫자, 공백만 유지)
def remove_special(text):
  return re.sub(r"[^가-힣a-zA-Z0-9\s]", "", text)

# 반복 문자 정규화
def normalize_repetition(text):
  return re.sub(r"(.)\1{2,}", r"\1\1", text)

# 오타 교정
def correct_korean(text):
  try:
    spelled = spell.check(text)
    return spelled.checked

  except:
    return text

# 전체 전처리 함수
def preprocessing_text(text):
  if pd.isna(text):
    return ""

  text = remove_emoji(text)
  text = remove_special(text)
  text = normalize_repetition(text)
  text = correct_korean(text)
  return text.strip()

In [ ]:
naver_df["preprocessing_content"] = naver_df["Content"].progress_apply(preprocessing_text)

100%|██████████| 67849/67849 [00:04<00:00, 16458.49it/s]


In [ ]:
naver_df.head()

,NO,product_name,Content,Info,category,Nutri,preprocessing_content
0,1.158627e+10,Bene,초코맛 사려다가 조금 더 저렴해서 샀는데 생각보다 맛있습니다. 물에 타먹었는데도 맛...,상품정보신고하기\n상품번호 11586271837 상품상태 신상품\n제조사 이선 브랜...,단백질 쉐이크,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g,초코맛 사려다가 조금 더 저렴해서 샀는데 생각보다 맛있습니다 물에 타먹었는데도 맛있...
1,1.158627e+10,Bene,맛있고 먹기 좋습니다,상품정보신고하기\n상품번호 11586271837 상품상태 신상품\n제조사 이선 브랜...,단백질 쉐이크,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g,맛있고 먹기 좋습니다
2,1.158627e+10,Bene,운동 전후 먹으면 든든하고 좋습니다.,상품정보신고하기\n상품번호 11586271837 상품상태 신상품\n제조사 이선 브랜...,단백질 쉐이크,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g,운동 전후 먹으면 든든하고 좋습니다
3,1.158627e+10,Bene,양이 많아서 좋아요 맛도 있어요.,상품정보신고하기\n상품번호 11586271837 상품상태 신상품\n제조사 이선 브랜...,단백질 쉐이크,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g,양이 많아서 좋아요 맛도 있어요
4,1.158627e+10,Bene,만족합니다 감사합니다,상품정보신고하기\n상품번호 11586271837 상품상태 신상품\n제조사 이선 브랜...,단백질 쉐이크,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g,만족합니다 감사합니다


In [ ]:
naver_df = naver_df[["NO", "product_name", "preprocessing_content", "Nutri", "category"]]

In [ ]:
naver_df.head()

,NO,product_name,preprocessing_content,Nutri,category
0,1.158627e+10,Bene,초코맛 사려다가 조금 더 저렴해서 샀는데 생각보다 맛있습니다 물에 타먹었는데도 맛있...,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g,단백질 쉐이크
1,1.158627e+10,Bene,맛있고 먹기 좋습니다,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g,단백질 쉐이크
2,1.158627e+10,Bene,운동 전후 먹으면 든든하고 좋습니다,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g,단백질 쉐이크
3,1.158627e+10,Bene,양이 많아서 좋아요 맛도 있어요,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g,단백질 쉐이크
4,1.158627e+10,Bene,만족합니다 감사합니다,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g,단백질 쉐이크


In [ ]:
naver_df.columns = [["Product_no", "Product_name", "Content", "Product_info", "category"]]

In [ ]:
naver_df.head()

,Product_no,Product_name,Content,Product_info,category
0,1.158627e+10,Bene,초코맛 사려다가 조금 더 저렴해서 샀는데 생각보다 맛있습니다 물에 타먹었는데도 맛있...,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g,단백질 쉐이크
1,1.158627e+10,Bene,맛있고 먹기 좋습니다,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g,단백질 쉐이크
2,1.158627e+10,Bene,운동 전후 먹으면 든든하고 좋습니다,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g,단백질 쉐이크
3,1.158627e+10,Bene,양이 많아서 좋아요 맛도 있어요,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g,단백질 쉐이크
4,1.158627e+10,Bene,만족합니다 감사합니다,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g,단백질 쉐이크


In [ ]:
naver_df.to_csv("/content/drive/MyDrive/데이터 수집/2.데이터 전처리/Preprocessing_naver.csv", index = False)

# 네이버 + 아마존 데이터

In [ ]:
amazon = pd.read_csv("/content/drive/MyDrive/데이터 수집/2.데이터 전처리/Preprocessing_amazon.csv")
amazon.head()

,Product_no,Product_name,Content,Product_info,category
0,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",okay so ive been incorporating these premier p...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
1,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",getting a little older and getting that protei...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
2,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",i have to say this is one of the best tasting ...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
3,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",product was delivered quickly but main box and...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
4,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",i really like the taste of premier protein ove...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질


In [ ]:
naver = pd.read_csv("/content/drive/MyDrive/데이터 수집/2.데이터 전처리/Preprocessing_naver.csv")
naver

,Product_no,Product_name,Content,Product_info,category
0,1.158627e+10,Bene,초코맛 사려다가 조금 더 저렴해서 샀는데 생각보다 맛있습니다 물에 타먹었는데도 맛있...,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g,단백질 쉐이크
1,1.158627e+10,Bene,맛있고 먹기 좋습니다,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g,단백질 쉐이크
2,1.158627e+10,Bene,운동 전후 먹으면 든든하고 좋습니다,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g,단백질 쉐이크
3,1.158627e+10,Bene,양이 많아서 좋아요 맛도 있어요,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g,단백질 쉐이크
4,1.158627e+10,Bene,만족합니다 감사합니다,주요 기능성(식약처인증) 해당 없음\n영양소 원료명(식약처고시) 단백질 단백질 28g,단백질 쉐이크
...,...,...,...,...,...
67844,8.701917e+09,Sentroom,딸한테 보내줬어요 화장실 가기편해요,주요 기능성(식약처인증) 체지방 감소 영양소 원료명(식약처고시) 크롬\n생균 100...,유산균
67845,8.701917e+09,Sentroom,비엔알17유산균이 좋다길래 먹어볼려고 찾다가 센트룸꺼도 있다는걸 알게 되었어요아직3...,주요 기능성(식약처인증) 체지방 감소 영양소 원료명(식약처고시) 크롬\n생균 100...,유산균
67846,8.701917e+09,Sentroom,화장실 잘 가네요 장건강에 도움 될거 같아요,주요 기능성(식약처인증) 체지방 감소 영양소 원료명(식약처고시) 크롬\n생균 100...,유산균
67847,8.701917e+09,Sentroom,화장실을 잘 못가서 기존에 잘 먹던 센트롬에서 유산균 도 나오길래 구매해보았슴니다 ...,주요 기능성(식약처인증) 체지방 감소 영양소 원료명(식약처고시) 크롬\n생균 100...,유산균


In [ ]:
amazon_naver = pd.concat([amazon, naver])
print(len(amazon_naver))

130597


In [ ]:
amazon_naver.to_csv("/content/drive/MyDrive/데이터 수집/2.데이터 전처리/네이버_아마존_통합.csv", index = False)

In [ ]:
amazon_naver

,Product_no,Product_name,Content,Product_info,category
0,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",okay so ive been incorporating these premier p...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
1,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",getting a little older and getting that protei...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
2,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",i have to say this is one of the best tasting ...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
3,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",product was delivered quickly but main box and...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
4,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",i really like the taste of premier protein ove...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
...,...,...,...,...,...
67844,8701916708.0,Sentroom,딸한테 보내줬어요 화장실 가기편해요,주요 기능성(식약처인증) 체지방 감소 영양소 원료명(식약처고시) 크롬\n생균 100...,유산균
67845,8701916708.0,Sentroom,비엔알17유산균이 좋다길래 먹어볼려고 찾다가 센트룸꺼도 있다는걸 알게 되었어요아직3...,주요 기능성(식약처인증) 체지방 감소 영양소 원료명(식약처고시) 크롬\n생균 100...,유산균
67846,8701916708.0,Sentroom,화장실 잘 가네요 장건강에 도움 될거 같아요,주요 기능성(식약처인증) 체지방 감소 영양소 원료명(식약처고시) 크롬\n생균 100...,유산균
67847,8701916708.0,Sentroom,화장실을 잘 못가서 기존에 잘 먹던 센트롬에서 유산균 도 나오길래 구매해보았슴니다 ...,주요 기능성(식약처인증) 체지방 감소 영양소 원료명(식약처고시) 크롬\n생균 100...,유산균


##### No컬럼 소수점 없애기 / Product_info컬럼 줄바꿈 문자 없애기

In [ ]:
amazon_naver["Product_no"] = amazon_naver["Product_no"].astype(str).apply(lambda x: re.sub(r"\.0$", "", x))

In [ ]:
amazon_naver

,Product_no,Product_name,Content,Product_info,category
0,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",okay so ive been incorporating these premier p...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
1,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",getting a little older and getting that protei...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
2,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",i have to say this is one of the best tasting ...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
3,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",product was delivered quickly but main box and...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
4,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",i really like the taste of premier protein ove...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
...,...,...,...,...,...
67844,8701916708,Sentroom,딸한테 보내줬어요 화장실 가기편해요,주요 기능성(식약처인증) 체지방 감소 영양소 원료명(식약처고시) 크롬\n생균 100...,유산균
67845,8701916708,Sentroom,비엔알17유산균이 좋다길래 먹어볼려고 찾다가 센트룸꺼도 있다는걸 알게 되었어요아직3...,주요 기능성(식약처인증) 체지방 감소 영양소 원료명(식약처고시) 크롬\n생균 100...,유산균
67846,8701916708,Sentroom,화장실 잘 가네요 장건강에 도움 될거 같아요,주요 기능성(식약처인증) 체지방 감소 영양소 원료명(식약처고시) 크롬\n생균 100...,유산균
67847,8701916708,Sentroom,화장실을 잘 못가서 기존에 잘 먹던 센트롬에서 유산균 도 나오길래 구매해보았슴니다 ...,주요 기능성(식약처인증) 체지방 감소 영양소 원료명(식약처고시) 크롬\n생균 100...,유산균


In [ ]:
# 줄바꿈 문자 제거
amazon_naver["Product_info"] = amazon_naver["Product_info"].astype(str).str.replace('\n', ' ', regex=False)

In [ ]:
amazon_naver

,Product_no,Product_name,Content,Product_info,category
0,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",okay so ive been incorporating these premier p...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
1,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",getting a little older and getting that protei...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
2,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",i have to say this is one of the best tasting ...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
3,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",product was delivered quickly but main box and...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
4,B07MJL8NXR,"Premier Protein Shake, Chocolate, 30g Protein ...",i really like the taste of premier protein ove...,"Packaged in a 11.5oz bottle, each Premier Prot...",단백질
...,...,...,...,...,...
67844,8701916708,Sentroom,딸한테 보내줬어요 화장실 가기편해요,주요 기능성(식약처인증) 체지방 감소 영양소 원료명(식약처고시) 크롬 생균 100억...,유산균
67845,8701916708,Sentroom,비엔알17유산균이 좋다길래 먹어볼려고 찾다가 센트룸꺼도 있다는걸 알게 되었어요아직3...,주요 기능성(식약처인증) 체지방 감소 영양소 원료명(식약처고시) 크롬 생균 100억...,유산균
67846,8701916708,Sentroom,화장실 잘 가네요 장건강에 도움 될거 같아요,주요 기능성(식약처인증) 체지방 감소 영양소 원료명(식약처고시) 크롬 생균 100억...,유산균
67847,8701916708,Sentroom,화장실을 잘 못가서 기존에 잘 먹던 센트롬에서 유산균 도 나오길래 구매해보았슴니다 ...,주요 기능성(식약처인증) 체지방 감소 영양소 원료명(식약처고시) 크롬 생균 100억...,유산균


In [ ]:
amazon_naver.to_csv("/content/drive/MyDrive/데이터 수집/2.데이터 전처리/네이버_아마존_통합.csv", index = False)